In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import unicodedata
import re
from bs4 import BeautifulSoup
from typing import Dict, List, Optional, Union, cast
from time import strftime, sleep

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')

import requests
from requests import get

from prepare import *
from acquire import *

from env import github_token, github_username

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hector/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/hector/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#run acquire.py

In [2]:
#reads in json
df = pd.read_json('data.json')
df.head()

,repo,language,readme_contents
0,thedaviddias/Front-End-Checklist,None,"<h1 align=""center"">\n<br>\n <img src=""https:/..."
1,onevcat/Kingfisher,Swift,"<p align=""center"">\n<img src=""https://raw.gith..."
2,FallibleInc/security-guide-for-developers,None,# 实用性 WEB 开发人员安全须知 \n\n### 目标读者 \n\n安全问题主要由以...
3,tailwindlabs/tailwindcss,JavaScript,"<p>\n <a href=""https://tailwindcss.com/"" ta..."
4,codepath/android_guides,None,# CodePath Android Cliffnotes\n\nWelcome to th...


In [3]:
df.shape

(560, 3)

In [4]:
df.language.value_counts(dropna = False)

JavaScript          141
Python               59
NaN                  54
Java                 46
TypeScript           45
Go                   36
C++                  30
C                    20
Swift                13
Shell                13
HTML                 12
Rust                 11
Kotlin                9
Ruby                  9
C#                    9
Vim script            6
PHP                   6
Jupyter Notebook      6
Vue                   6
CSS                   5
Objective-C           3
CoffeeScript          2
TeX                   2
Haskell               2
Lua                   2
Elixir                2
OCaml                 1
Rascal                1
Dockerfile            1
Crystal               1
Standard ML           1
Batchfile             1
Dart                  1
Makefile              1
AsciiDoc              1
Clojure               1
Emacs Lisp            1
Name: language, dtype: int64

In [5]:
#sums null counts
df.isnull().sum()

repo                0
language           54
readme_contents     0
dtype: int64

In [6]:
#drops nulls
df = df.dropna()
df.shape

(506, 3)

In [7]:
#reset index 
df = df.reset_index(drop = True)
df.head()

,repo,language,readme_contents
0,onevcat/Kingfisher,Swift,"<p align=""center"">\n<img src=""https://raw.gith..."
1,tailwindlabs/tailwindcss,JavaScript,"<p>\n <a href=""https://tailwindcss.com/"" ta..."
2,github/fetch,JavaScript,# window.fetch polyfill\n\nThe `fetch()` funct...
3,ianstormtaylor/slate,TypeScript,"<p align=""center"">\n <a href=""#""><img src=""./..."
4,Kong/insomnia,JavaScript,# Insomnia REST Client\n\n[![Slack Channel](ht...


In [8]:
def clean(text):
#this takes in the readme_content and prepares it.
#the result is a clean value
    #text = df['readme_contents'].iloc[0] #takes in the first value of the 1st row of the readme_contents column and assigns it 'text' variable
    text = text.lower()#takes in the variable and sets all to lower
    soup = BeautifulSoup(text, 'html.parser')#soups it!
    text = soup.get_text()#grabs the texts from the readme_contents
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')#normalized and encodes
    text = re.sub(r"[^a-z0-9'\s]", '', text)#keeps alpha numeric characters
    wnl = nltk.stem.WordNetLemmatizer()#lemma
    lemmas = [wnl.lemmatize(word) for word in text.split()]#lemma loop
    text_lemma = ' '.join(lemmas)#lemmas
    stopwords = nltk.corpus.stopwords.words('english')#stopword
    newStopWords = ['u','ha','wa']#we can change these. I left it in from my last exercise as a place holder if we found any other stopwords we wanted to use.
    stopwords.extend(newStopWords)#adds new stopwords
    words = text_lemma.split()#splits
    filtered_words = [w for w in words if w not in stopwords]#loops the split
    speech = ' '.join(filtered_words)#joins it all thgether

#Could not figure out how to loop this correctly
    return speech

In [10]:
#assigns 'clean' column to the df and calls the 'clean' function to iterate through the rows.
df['clean'] = df.readme_contents.apply(clean)
df.head()

,repo,language,readme_contents,clean
0,onevcat/Kingfisher,Swift,"<p align=""center"">\n<img src=""https://raw.gith...",kingfisher powerful pureswift library download...
1,tailwindlabs/tailwindcss,JavaScript,"<p>\n <a href=""https://tailwindcss.com/"" ta...",utilityfirst cs framework rapidly building cus...
2,github/fetch,JavaScript,# window.fetch polyfill\n\nThe `fetch()` funct...,windowfetch polyfill fetch function promisebas...
3,ianstormtaylor/slate,TypeScript,"<p align=""center"">\n <a href=""#""><img src=""./...",completely customizable framework building ric...
4,Kong/insomnia,JavaScript,# Insomnia REST Client\n\n[![Slack Channel](ht...,insomnia rest client slack channelhttpschatins...


In [ ]:
#### Notes below. Some possible useful code to get rid of the chinese documents below....

In [ ]:
#This gets rid of the chinese documents
def is_chinese(texts):
    '''
    This function takes in a dataframe and return true if the scanned text is in chinese
    '''
    if re.search("[\u4e00-\u9FFF]", texts):
            return True

In [ ]:
#drops true values from is_chinese function
df = df[df.readme_contents.apply(is_chinese) !=True]

In [ ]:
df = prepare_nlp_data(df, content = 'readme_contents', extra_words=['customer', 'customers', '1', '2'])

In [ ]:
df.head()